<a href="https://colab.research.google.com/github/dipnarayan501/CNN/blob/main/DL_CNN_Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PART - B Fine-Tuning a Pre-Trained model**

Import Google colab to drive 

In [ ]:
#Mounting to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import shutil and splitting the dataset 

In [ ]:
import shutil  # provides the ability to copy and removal of files
import os 
import random
try:
    shutil.rmtree("inaturalist_12K")
except:
    pass

# Unzip the natural datset     
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip

try:
    os.remove("nature_12K.zip")
except:
    pass
try:
    shutil.rmtree("inaturalist_12K/validation/")
except:
    pass

# dataset directory 
temp = os.listdir("inaturalist_12K/train")
class_names = [name for name in temp if name != ".DS_Store"] 

#Spliting valdiation dataset 
validation_split = 0.1 #splitting dataset 10%
def spliting_train_data(validation_split = 0.1):
  os.mkdir("inaturalist_12K/validation/")
  for each_class in class_names:
      os.mkdir("inaturalist_12K/validation/"+each_class)
      train_images = os.listdir("inaturalist_12K/train/"+each_class)
      train_images_per_class = list(train_images)
      #print(len(train_images_per_class))
      random.shuffle(train_images_per_class) # Shuffle the training images

      validation_images_per_class = train_images_per_class[:round(validation_split*len(train_images_per_class))]
     # print(len(validation_images_per_class))

      for im in validation_images_per_class:
          shutil.move("inaturalist_12K/train/"+each_class+"/"+im, "inaturalist_12K/validation/"+each_class+"/"+im)
spliting_train_data(validation_split)


Exploring the Data

In [ ]:
train = 0
val = 0
test = 0

for each_class in class_names:
      train_images = os.listdir("inaturalist_12K/train/"+each_class)
      train+=len(list(train_images))

      validation_images = os.listdir("inaturalist_12K/validation/"+each_class)
      
      val+=len(list(validation_images))

      test_images = os.listdir("inaturalist_12K/val/"+each_class)
      
      test+=len(list(test_images))
 # print training validation test      
print("training" , train)
print("validation Examples = ", val)
print("test examples = ", test)

training 9000
validation Examples =  1000
test examples =  2000


Import required libriaries for pre trained module 

In [ ]:
import os  
import glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization


# implement weights and biases 

!pip install wandb
import wandb
from keras.callbacks import EarlyStopping, ModelCheckpoint
from wandb.keras import WandbCallback
from PIL import Image
# image plotting 
%matplotlib inline
import matplotlib.pyplot as plt

     |████████████████████████████████| 1.7 MB 8.2 MB/s 
     |████████████████████████████████| 181 kB 56.3 MB/s 
     |████████████████████████████████| 144 kB 54.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ddb10b4a79c22c064e9a05520ada3ea35a8e57d32f05145eb0a283fff8fb4a44
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


Setup the Train Test and Validation generators


In [ ]:
data_dir = "inaturalist_12K"
#data_augmentation = True
# data prepartion 
def data_preparation(data_dir , data_augmentation , batch_size):
  train_dir = os.path.join(data_dir, "train")
  val_dir = os.path.join(data_dir, "validation")
  test_dir = os.path.join(data_dir, "val")
  if data_augmentation == True:
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      height_shift_range=0.2,
                                      width_shift_range=0.2,
                                      horizontal_flip=True,
                                      zoom_range=0.2,
                                      fill_mode="nearest")
  else:
      train_datagen = ImageDataGenerator(rescale=1./255)
  val_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1./255)


  train_generator = train_datagen.flow_from_directory(
          train_dir,
          target_size=(224, 224),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
          val_dir,
          target_size=(224, 224),
          shuffle=True,
          color_mode="rgb",
          batch_size=batch_size,
          class_mode='categorical')
  
  test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size= (224, 224),
        color_mode="rgb",
        batch_size=batch_size,
        class_mode='categorical')

  
  
  return train_generator , val_generator, test_generator
#train_generator , val_generator, test_generator = data_preparation(data_dir , data_augmentation = True , batch_size = 250)

Implement pre trained InceptionV3, InceptionResNetV2, ResNet50, Xception

In [ ]:
def pretrain_model(pretrained_model_name, dropout, dense_layer, pre_layer_train=None):
    
  # keras sequential model based on a pre-trained model intended to be fine tuned
  

    # shape image size 
    shape = (224 ,224, 3)

    # adding a pretrained model without the top dense layer
    if pretrained_model_name == 'ResNet50':
      pretrained_model = tf.keras.applications.ResNet50(include_top=False,input_shape=shape, weights='imagenet')
    elif pretrained_model_name == 'InceptionV3':
      pretrained_model = tf.keras.applications.InceptionV3(include_top=False,input_shape= shape, weights='imagenet')    
    elif pretrained_model_name == 'Xception':
      pretrained_model = tf.keras.applications.Xception(include_top=False,input_shape= shape, weights='imagenet')
    elif pretrained_model_name == 'InceptionResNetV2':
      pretrained_model = tf.keras.applications.InceptionResNetV2(include_top=False,input_shape= shape, weights='imagenet')
    
    else:
      raise Exception('no pretrained model given')
      

    #freeze all layers
    for layer in pretrained_model.layers:
        layer.trainable=False 
    
    #setting  top layers as trainable
    if pre_layer_train:
      for layer in pretrained_model.layers[-pre_layer_train:]:
        layer.trainable=True

    model = tf.keras.models.Sequential()
    #adding pretrained model
    model.add(pretrained_model)
    # flatten layer  
    model.add(Flatten()) 
    #Adding a dense layer
    model.add(Dense(dense_layer, activation= 'relu'))
    #Adding dropout
    model.add(Dropout(dropout))
    #output layer with softmax  
    model.add(Dense(10, activation="softmax"))
 
    return model

Configuration wandb


In [ ]:
def train():
    # Default values for hyper-parameters
    config_defaults = {
        "data_augmentation": True,
        "batch_size": 250,
        "dropout": 0.1,
        "dense_layer": 256,
        "learning_rate": 0.0001,
        "epochs": 5,
        "pre_layer_train": None,
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # Local variables, values obtained from wandb config
    data_augmentation = config.data_augmentation
    batch_size = config.batch_size
    dropout = config.dropout
    dense_layer = config.dense_layer
    learning_rate = config.learning_rate
    epochs = config.epochs
    pre_layer_train = config.pre_layer_train

    
   # Display the hyperparameters
    run_name = "pre_train_mdl_{}_aug_{}_bs_{}_ep_{}_dropout_{}_dense_{}".format(pre_train_model, data_augmentation, batch_size,epochs, dropout, dense_layer)
    print(run_name)

    # Create the data generators
    train_generator , val_generator, test_generator = data_preparation(data_dir , data_augmentation = data_augmentation, batch_size = batch_size)
    
    # Define the model
    model = pretrain_model(pretrained_model_name = pre_train_model, dropout = dropout, dense_layer = dense_layer, pre_layer_train=pre_layer_train)
    print(model.count_params())


    model.compile(optimizer=Adam(learning_rate = learning_rate), loss = 'categorical_crossentropy', metrics = ['accuracy'])

    # Early Stopping callback
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

    # To save the model with best validation accuracy
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
    #Training the model
    history = model.fit(train_generator,
                        steps_per_epoch = train_generator.n//train_generator.batch_size,
                        validation_data = val_generator,
                        validation_steps = val_generator.n//val_generator.batch_size,
                        epochs=epochs, verbose = 2, 
                        callbacks=[WandbCallback(), earlyStopping, checkpoint])
      

    

    
    # Meaningful name for the run
    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()
    return history

sweeps configuration for hyper parameters 

In [ ]:

sweep_config = {
  "name": "CNN_PartB",
  "metric": {
      "name":"val_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "data_augmentation": {
            "values": [True, False]
        },
        "batch_size": {
            "values": [128, 256]
        },
        "learning_rate": {
            "values": [0.001, 0.0001]
        },
        "epochs": {
            "values": [5, 10]
        },
        "dropout": {
            "values": [0, 0.2,0.4]
        },
        "dense_layer": {
            "values": [128, 256, 512]
        },
          "pre_layer_train": {
            "values": [None, 10, 20]
        },

      
           }
}


# Generates a sweep id
sweep_id = wandb.sweep(sweep_config, entity="moni6264", project="test_cnn_partb")

Running wandb

In [ ]:
#Note:  You might get different result because of shuffling of dataset
pre_train_model = "ResNet50'"   #you can change model for wandb sweeps
sweep_id = "y68teyjh"
wandb.agent(sweep_id, train,entity="moni6264", project="test_cnn_partb", count=10)